<a href="https://colab.research.google.com/github/singhharshvardhan681-sys/college-study-planner/blob/main/College_Study_Planner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import os
from datetime import datetime

# The name of the file used to store tasks persistently
TASK_FILE = 'tasks.json'

class StudyPlanner:
    """
    Manages study tasks, including adding, viewing, and updating task status,
    with automatic saving and loading to a JSON file.
    """
    def __init__(self):
        # List of task dictionaries
        self.tasks = []
        self._load_tasks()

    def _load_tasks(self):
        """Loads tasks from the persistent JSON file."""
        if os.path.exists(TASK_FILE):
            try:
                with open(TASK_FILE, 'r') as f:
                    self.tasks = json.load(f)
            except json.JSONDecodeError:
                print(f"Warning: Could not decode {TASK_FILE}. Starting with an empty planner.")
                self.tasks = []
        else:
            print("No previous task file found. Starting a new planner.")

    def _save_tasks(self):
        """Saves the current list of tasks to the JSON file."""
        with open(TASK_FILE, 'w') as f:
            json.dump(self.tasks, f, indent=4)

    def add_task(self, name, subject, deadline_str, priority_level):
        """
        Adds a new task to the planner.

        Args:
            name (str): The name of the task (e.g., "Write Chapter 1").
            subject (str): The associated college subject.
            deadline_str (str): The deadline in 'YYYY-MM-DD' format.
            priority_level (str): The priority (e.g., 'High', 'Medium', 'Low').
        """
        try:
            # Validate deadline format
            datetime.strptime(deadline_str, '%Y-%m-%d')

            new_task = {
                'id': self._generate_id(),
                'name': name,
                'subject': subject,
                'deadline': deadline_str,
                'priority': priority_level,
                'completed': False,
                'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            self.tasks.append(new_task)
            self._save_tasks()
            print(f"\n✅ Task '{name}' added successfully.")
        except ValueError:
            print("\n❌ Invalid date format. Please use YYYY-MM-DD.")

    def _generate_id(self):
        """Generates a simple incremental ID for tasks."""
        return max([t['id'] for t in self.tasks] + [0]) + 1

    def view_tasks(self, sort_key='deadline', hide_completed=True):
        """
        Displays all tasks, optionally sorted and filtered.

        Args:
            sort_key (str): Key to sort by ('deadline' or 'priority').
            hide_completed (bool): If True, completed tasks are hidden.
        """
        display_tasks = self.tasks

        if hide_completed:
            display_tasks = [t for t in display_tasks if not t['completed']]

        if not display_tasks:
            print("\n--- No tasks to display! Great job, or time to add some projects. ---")
            return

        # Sort tasks
        if sort_key == 'deadline':
            # Sort by deadline, completed status (uncompleted first), and priority
            display_tasks.sort(key=lambda t: (t['completed'], t['deadline']))
        elif sort_key == 'priority':
            # Define sort order for priority
            priority_order = {'High': 0, 'Medium': 1, 'Low': 2}
            display_tasks.sort(key=lambda t: (t['completed'], priority_order.get(t['priority'], 3)))

        print("\n--- College Study Planner Tasks ---")
        for i, task in enumerate(display_tasks, 1):
            status = "✅ COMPLETE" if task['completed'] else "⏳ PENDING"
            print(f"[{i:02d}] ID:{task['id']} | Status: {status}")
            print(f"      Name: {task['name']}")
            print(f"      Subject: {task['subject']} | Priority: {task['priority']}")
            print(f"      Due: {task['deadline']}")
        print("---------------------------------")
        return display_tasks

    def mark_complete(self, display_index):
        """Marks a task as completed based on its displayed index."""
        try:
            # We need to re-generate the displayed list to map index to actual task
            displayed_tasks = self.view_tasks(hide_completed=True)
            if display_index > 0 and display_index <= len(displayed_tasks):
                task_to_update = displayed_tasks[display_index - 1]

                # Find the actual task object in the main list using its ID
                original_task = next(t for t in self.tasks if t['id'] == task_to_update['id'])

                original_task['completed'] = True
                self._save_tasks()
                print(f"\n🎉 Task '{original_task['name']}' marked as COMPLETE!")
            else:
                print("\n❌ Invalid task number.")
        except (ValueError, IndexError):
            print("\n❌ Invalid input. Please enter a number from the displayed list.")
        except StopIteration:
             print("\n❌ Error finding task ID in original list.")

    def delete_task(self, display_index):
        """Deletes a task based on its displayed index."""
        try:
            # We need to re-generate the displayed list to map index to actual task
            displayed_tasks = self.view_tasks(hide_completed=False)
            if display_index > 0 and display_index <= len(displayed_tasks):
                task_to_delete = displayed_tasks[display_index - 1]

                # Filter out the task by its ID
                self.tasks = [t for t in self.tasks if t['id'] != task_to_delete['id']]

                self._save_tasks()
                print(f"\n🗑️ Task '{task_to_delete['name']}' deleted.")
            else:
                print("\n❌ Invalid task number.")
        except (ValueError, IndexError):
            print("\n❌ Invalid input. Please enter a number from the displayed list.")


def main():
    """Main function to run the command-line interface."""
    planner = StudyPlanner()

    while True:
        print("\n==============================")
        print("    College Project Planner")
        print("==============================")
        print("1. Add New Task/Project")
        print("2. View Pending Tasks (Sorted by Deadline)")
        print("3. View All Tasks (Including Completed)")
        print("4. Mark Task as Complete")
        print("5. Delete Task/Project")
        print("6. Exit")

        choice = input("Enter your choice (1-6): ").strip()

        if choice == '1':
            print("\n--- Add New Task ---")
            name = input("Task Name: ").strip()
            subject = input("Subject/Course: ").strip()

            # Simple input loop for date validation
            while True:
                deadline_str = input("Deadline (YYYY-MM-DD): ").strip()
                try:
                    datetime.strptime(deadline_str, '%Y-%m-%d')
                    break
                except ValueError:
                    print("Invalid date format. Please use YYYY-MM-DD.")

            priority_map = {'1': 'High', '2': 'Medium', '3': 'Low'}
            while True:
                priority_choice = input("Priority (1: High, 2: Medium, 3: Low): ").strip()
                if priority_choice in priority_map:
                    priority = priority_map[priority_choice]
                    break
                else:
                    print("Invalid priority choice.")

            planner.add_task(name, subject, deadline_str, priority)

        elif choice == '2':
            # View pending tasks, sorted by deadline (default behavior)
            planner.view_tasks(sort_key='deadline', hide_completed=True)

        elif choice == '3':
            # View all tasks
            print("Viewing ALL tasks. Completed tasks are marked.")
            planner.view_tasks(sort_key='deadline', hide_completed=False)

        elif choice == '4':
            # Mark complete
            tasks_to_complete = planner.view_tasks(hide_completed=True)
            if tasks_to_complete:
                try:
                    index = int(input("Enter the [index] number of the task to mark complete: ").strip())
                    planner.mark_complete(index)
                except ValueError:
                    print("Invalid input. Please enter a valid number.")

        elif choice == '5':
            # Delete task (shows all tasks, including completed)
            tasks_to_delete = planner.view_tasks(hide_completed=False)
            if tasks_to_delete:
                try:
                    index = int(input("Enter the [index] number of the task to DELETE: ").strip())
                    planner.delete_task(index)
                except ValueError:
                    print("Invalid input. Please enter a valid number.")

        elif choice == '6':
            print("\nExiting Planner. Tasks saved to tasks.json.")
            break

        else:
            print("\nInvalid choice. Please enter a number between 1 and 6.")


if __name__ == "__main__":
    main()

No previous task file found. Starting a new planner.

    College Project Planner
1. Add New Task/Project
2. View Pending Tasks (Sorted by Deadline)
3. View All Tasks (Including Completed)
4. Mark Task as Complete
5. Delete Task/Project
6. Exit

--- Add New Task ---

✅ Task 'english' added successfully.

    College Project Planner
1. Add New Task/Project
2. View Pending Tasks (Sorted by Deadline)
3. View All Tasks (Including Completed)
4. Mark Task as Complete
5. Delete Task/Project
6. Exit

--- College Study Planner Tasks ---
[01] ID:1 | Status: ⏳ PENDING
      Name: english
      Subject: english | Priority: Low
      Due: 2025-11-25
---------------------------------

    College Project Planner
1. Add New Task/Project
2. View Pending Tasks (Sorted by Deadline)
3. View All Tasks (Including Completed)
4. Mark Task as Complete
5. Delete Task/Project
6. Exit

--- College Study Planner Tasks ---
[01] ID:1 | Status: ⏳ PENDING
      Name: english
      Subject: english | Priority: Low
   